In [1]:
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
# 創建時間窗口的訓練集數據
def create_dataset(closing_prices, time_window):
    features,targets = [],[]
    for i in range(len(closing_prices) - time_window):
        features.append(closing_prices.iloc[i:i+time_window])
        targets.append(closing_prices.iloc[i+time_window])   
    return np.array(features),np.array(targets)

In [5]:
# dataset = pd.read_csv(f'../Data/BTCUSD-all.csv')
# # dataset= np.log(dataset["Close"])
# train_data, test_data = train_test_split(dataset["Close"], test_size=7, shuffle=False)

# # Take the logarithm
# # train_data = np.log(train_data)
# # test_data= np.log(test_data)

# # Calculate mean and standard deviation
# mean = np.mean(train_data)
# std = np.std(train_data)*3

# # Z-score normalization
# standardized_train = (train_data - mean) / std
# standardized_test=(test_data- mean)/ std
# print(train_data.shape,test_data.shape)

In [15]:
data = pd.read_csv(f'../Data/BTCUSD-all.csv') 
train_data, test_data = train_test_split(data['Close'], test_size=0.2, shuffle=False)

lb=12
trainX, trainY = create_dataset(train_data, lb)
testX, testY = create_dataset(test_data, lb)
tscv = TimeSeriesSplit(n_splits=5)
best_model = None
best_rmse = np.inf

# for train_index, val_index in tscv.split(trainY):
#     train_val_data, val_data = trainY[train_index], trainY[val_index]    
#     model = auto_arima(train_val_data,exogenous=trainX[train_index],max_p=20,max_q=20,seasonal=False)   
#     forecast = model.predict(len(val_data))
#     rmse = np.sqrt(mean_squared_error(val_data, forecast))
#     if rmse < best_rmse:
#         best_rmse = rmse
#         best_model = model
# best_model.fit(trainY)

best_model = auto_arima(trainY,exogenous=trainX,max_p=20,max_q=20,seasonal=False)
best_model.fit(trainY)
best_model_params = best_model.get_params()['order']
print("最佳模型參數:ARIMA", best_model_params)


# forecast
test_forecast = best_model.predict(n_periods=len(testY))


test_data_mae=mean_absolute_error(testY,test_forecast)
test_data_rmse=mean_squared_error(testY,test_forecast)**0.5        
      

# predict train
train_forecast = best_model.predict_in_sample()    

train_data_mae=mean_absolute_error(trainY,train_forecast)
train_data_rmse=mean_squared_error(trainY,train_forecast)**0.5


print(f'MAE train:{train_data_mae:.4f} test:{test_data_mae:.4f}')   
print(f'RMSE train:{train_data_rmse:.4f} test:{test_data_rmse:.4f}')         
        
        

最佳模型參數:ARIMA (0, 1, 0)
MAE train:322.4943 test:10901.0974
RMSE train:767.7009 test:12242.3494


In [11]:
best_model.

ValueError: Input contains NaN.